Importing the panda and numpy libraries

In [1]:
import numpy as np
import pandas as pd

Creating the list of flight duration between different airports and gates. Also creating the directory for minimum ground time at each airport gate

In [2]:
flight_time =[['AUS1','DAL1',50],['AUS1','DAL2',50],['DAL1','AUS1',50],['DAL2','AUS1',50],['AUS1','HOU1',45],['AUS1','HOU2',45],['AUS1','HOU3',45],['HOU1','AUS1',45],['HOU2','AUS1',45],['HOU3','AUS1',45],['DAL1','HOU1',65],['DAL1','HOU2',65],['DAL1','HOU3',65],['DAL2','HOU1',65],['DAL2','HOU2',65],['DAL2','HOU3',65],['HOU1','DAL1',65],['HOU1','DAL2',65],['HOU2','DAL1',65],['HOU2','DAL2',65],['HOU3','DAL1',65],['HOU3','DAL2',65]]
ground_time = {'AUS1':25, 'DAL1':30,'DAL2':30, 'HOU1':35,'HOU2':35,'HOU3':35}
gates = ['AUS1','DAL1','DAL2','HOU1','HOU2','HOU3']

creating function to convert the minutes from midnight to military time

In [3]:
def military_time(minutes):
    hr = minutes // 60
    minute = minutes % 60
    time = str(hr).zfill(2) + str(minute).zfill(2)
    return(time)

Assigning the initial flights

In [4]:
flight_schedule = [['T1','HOU1','AUS1',360,360,360],
                   ['T2','HOU2','DAL1',360,360,360],
['T3','HOU3','DAL2',360,360,360],
['T4','AUS1','HOU1',360,360,360],
['T5','DAL1','HOU2',360,360,360],
['T6','DAL2','HOU3',360,360,360]]

creating the pandas data frame to store the schedule of flights

In [5]:
d2={'flight':None,'d_a':None,'a_a':None,'d_t':None,'a_t':None,'n_dt':None}
index2=np.arange(0,70,1)
ms= pd.DataFrame(d2,index2)
ms.to_string()
count3=6
for i in range(0,6):
    ms.loc[i,:]=flight_schedule[i]
freq = {'AUS1':1,'DAL1':1,'DAL2':1,'HOU1':1,'HOU2':1,'HOU3':1}
avaibility_matrix = pd.DataFrame('0',index=range(360,1501),columns=['AUS1','DAL1','DAL2','HOU1','HOU2','HOU3'])

creating panda dataframe to check the gate avaibility based on initial flight schedule using python list. The dataframe  has minutes from midnight as a index and airport gates as a column.

Loop to schedule flight based on avaibility of gate for next arrival time at that gate. 

In [6]:
try:
    signal = 'green'
    for i in range(1,14):
            if signal == 'green':
                flight_schedule.sort(key = lambda x:x[5])
                flight_code = 'green'
                for flight in flight_schedule:
                    if flight_code == 'green':
                        code = 'green'
                        while code == 'green':
                            temp_dtime = flight[5]
                            dept_city = flight[2]
                            gate_req = []
                            for f_time in flight_time:
                                loc = flight_time.index(f_time)
                                if flight_time[loc][0] == dept_city:
                                        fly_time = flight_time[loc][2]
                                        x = flight_time[loc][1]
                                        arrival_time = temp_dtime + fly_time
                                        ndept_time = arrival_time + ground_time[x]
                                        frequency = freq[x]
                                        gate_req.append([x,arrival_time,ndept_time,fly_time,frequency])
                            gate_req.sort(key = lambda x:(x[2],x[4]))
                            count = 1

                            for i in gate_req:
                                flag=0
                                a = i[1]
                                b = i[2]
                                c = i[0]
                                d = i[3]
                                if a > 1320:
                                            code = 'red'
                                            flight_code = 'red'
                                            signal = 'red'
                                            break
                                else:
                                    for q in range(a,b):
                                        if avaibility_matrix.loc[q,c] != '0':
                                            flag=1
                                            break
                                    if flag==1:
                                        if len(gate_req) == count:
                                            flight[5] += 1
                                            break
                                        else:
                                            count += 1
                                            continue
                                    loc1 = flight_schedule.index(flight)
                                    tail = flight_schedule[loc1][0]
                                    flight_schedule[loc1][1] = dept_city
                                    flight_schedule[loc1][2] = c
                                    flight_schedule[loc1][3] = a - d
                                    flight_schedule[loc1][4] = a
                                    flight_schedule[loc1][5] = b
                                    freq[c] += 1
                                    if a <= 1320:
                                        ms.loc[count3,:]=flight_schedule[loc1]
                                        ms=ms.dropna()
                                        for  index, row in avaibility_matrix.iterrows():        
                                            for index1, row1 in ms.iterrows():
                                                        if int(index) >= int(row1['flight']) and int(index) <= int(row1['n_dt']):
                                                            avaibility_matrix.loc[index,row1['d_a']] = row1['a_a']
                                    else:
                                        signal = 'red'
                                        for index,row in avaibility_matrix.iterrows():
                                            if index > a:
                                                avaibility_matrix.loc[index,dept_city] = tail

                                    count3=count3+1
                                    code = 'red'
                                    break

except:
    print('day is over')

Drop any null columns from ms dataframe

In [7]:
ms.drop(ms.index[:6], inplace=True)

In [8]:
ms.to_csv('gate_schedule.csv', index=False)

convert the columns to display airport name instead of gate name

In [9]:
departure_airport = ms.a_t.str[:3]
arrival_airport = ms.d_a.str[:3]
ms['origin'] = departure_airport
ms['destination'] = arrival_airport

dropping the gate name columns and unnecessary column to prepare the data frame for final output

In [10]:
ms.drop('a_t',axis=1,inplace = True)
ms.drop('d_a',axis=1,inplace = True)
ms.drop('n_dt',axis=1,inplace = True)

rearranging the columns for final output

In [11]:
ms = ms[['a_a','origin', 'destination', 'd_t', 'flight']]

renaming the columns as per output file requirement

In [12]:
ms.columns = ['tail_number','origin', 'destination', 'departure_time', 'arrival_time']

converting the time from minutes to midnight to military time 

In [13]:
for index, row in ms.iterrows():
    x1=military_time(row['departure_time'])
    x2= military_time(row['arrival_time'])
    
    ms.loc[index,'departure_time']=x1
    ms.loc[index,'arrival_time']=x2


saving the final output in csv file

In [14]:
ms.to_string
final_schedule = ms.sort_values(['tail_number','departure_time'])
final_schedule.to_csv('flight_schedule.csv', index=False)